In [2]:
from datetime import datetime, timedelta
import psycopg2
import json
import pandas as pd
from psycopg2.extras import execute_values
import numpy as np
import os
import requests
import http.client
import os
import glob

In [3]:
from sqlalchemy import create_engine, update,insert, MetaData, Table,String, Column, Integer, Date, Boolean, ForeignKey
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

### Credenciales

In [4]:
path = os.curdir

In [5]:
#API FUTBOL
with open(f"{path}/keys/credencial_futbol.txt",'r') as file:
    api_key = file.read().strip()

# CONEXION BBDD

with open(f"{path}/keys/credencial_bbdd.json") as file:
    credenciales = json.load(file)

host= credenciales["host"]
schema=credenciales["database"]
usuario= credenciales["user"]
password= credenciales["password"]

### Funciones

In [6]:
import pymysql
pymysql.install_as_MySQLdb()

def consultaSQL(query,usuario,password,schema,host):
    conn = pymysql.connect(host=host,user = usuario, password=password, database=schema)  # Conectar a la base de datos
    
    cursor = conn.cursor(pymysql.cursors.DictCursor)  # Crear un cursor
    cursor.execute(query)  # Ejecutar una consulta

    if query.upper().startswith('SELECT') or query.upper().startswith('SHOW'):
        data = cursor.fetchall()  # Traer los resultados de un select
    else:
        conn.commit()  # Hacer efectiva la escritura de datos
        data = None
        cursor.close()  # Cerrar el cursor
        conn.close()  # Cerrar la conexion
    return data

### Conexión BBD

### Consulta API

In [7]:
def conectar_api():
    conn_api = http.client.HTTPSConnection("v3.football.api-sports.io")
    date = datetime.now()
    headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': api_key
    }

    return (conn_api, headers, date)


def request_api(endpoint):
    try:
        conn_api, headers, date = conectar_api()
        conn_api.request("GET", endpoint, headers=headers)
        res = conn_api.getresponse()
        if res.status == 200:
            data = res.read()
            respuesta_json =json.loads(data)
            
            if len(respuesta_json["errors"]) != 0:
                for clave, valor in respuesta_json["errors"].items():
                    print(f"Se ha generado error de tipo: {clave} - {valor}")
                return(respuesta_json["errors"])
            else: 
                print("Consulta exitosa")
                endpoint_limpio = endpoint.replace('/','_')+'_'
                endpoint_limpio = endpoint.replace('?','_')+'_'
                with open(path+'/'+"data_cruda/"+endpoint_limpio+str(date.year)+'-'+str(date.month)+'-'+str(date.day)+'-'+str(date.hour)+".json", "w") as json_file:
                            json.dump(respuesta_json, json_file)
                return(respuesta_json)
        else:
            print("Error en la solicitud a la API. Código de estado: ", res.status)
    except http.client.HTTPException as e:
        respuesta = "Error de conexión: "+ e
        print(respuesta)
        return(respuesta)


In [8]:
'''
Endpoints
/leagues
/teams
/teams/statics
/teams/seasons
/teams/countries
'''

'\nEndpoints\n/leagues\n/teams\n/teams/statics\n/teams/seasons\n/teams/countries\n'

##### Definición Tablas Metadata

In [9]:
from metadata_tablas import creacion_meta_data, creacion_tablas_metadata
session, metadata = creacion_meta_data(usuario,password,schema)
paises, ligas, datos_ligas, equipos,ligas_equipos, equipos_estadisticas, tarjetas_detalles, goles_detalles, hitos_equipos, formaciones_equipos = creacion_tablas_metadata(metadata)


In [10]:
def insertar_base(query):
    session.execute(query)

    # Confirmar los cambios en la base de datos
    session.commit()

    # Cerrar la sesión
    session.close()


Fragmento codigo que llama a endpoint countries, trae paises e inserta en tabla paises de la base futbol

In [11]:
#endpoint = "/countries"
#respuesta= request_api(endpoint)
#paises_api = respuesta["response"]  

#### Consulta Paises

In [12]:
# PAISES
from sqlalchemy.dialects.mysql import insert as mysql_insert
import time
def consulta_paises(intento = 0):
    ''' Funcion recibe un valor de intento, en caso de fallar intenta hasta 3 veces'''
    
    intentos = 3 - intento
    if intentos >0:
        try:
            endpoint = "/countries"
            respuesta= request_api(endpoint)
            paises_api = respuesta["response"]       
        
            for pais_api in paises_api:
                if pais_api["name"] == "World":
                    pais_api["code"] = "WM"
                try:
                    ins = mysql_insert(paises).values(codigo=pais_api["code"],pais= pais_api["name"])
                    ins = ins.on_duplicate_key_update(codigo=ins.inserted.codigo)  # Para MySQL, especificar que hacer en caso de duplicado
                    session.execute(ins)
                    session.commit()
                    session.close()
                except Exception as e:
                    # e es la variable que almacena información sobre la excepción ocurrida
                    print("Mensaje de error:", e)    
                    continue
            # Confirmar los cambios y cerrar la sesión

        except Exception as e:
            print("intento fallido")
            print("Mensaje de error:", e)
            time.sleep(0.5)
            consulta_paises(intento+1)
        finally:
        # Cerrar la sesión
            return

#### Consultas Ligas

In [13]:
def consulta_descripcion_ligas_api(response):
    with engine.connect() as con:
        for liga in response:
            liga_id=liga['league']['id'],
            for season in liga["seasons"]:
                datos_ins = insert(datos_ligas).values(
                liga_id=liga_id[0],
                temporada=season['year'],
                inicio=season['start'],
                fin=season['end'],
                f_eventos=season['coverage']['fixtures']['events'],
                f_alineaciones=season['coverage']['fixtures']['lineups'],
                f_estadisticas=season['coverage']['fixtures']['statistics_fixtures'],
                f_jugadores=season['coverage']['fixtures']['statistics_players'],
                clasificaciones=season['coverage']['standings'],
                jugadores=season['coverage']['players'],
                top_goleadores=season['coverage']['top_scorers'],
                top_asistencias=season['coverage']['top_assists'],
                top_tarjetas=season['coverage']['top_cards'],
                lesiones=season['coverage']['injuries'],
                predicciones=season['coverage']['predictions'],
                apuestas=season['coverage']['odds']
            ).prefix_with("IGNORE")  # Ignorar inserción si hay duplicados en clave primaria
            con.execute(datos_ins)
        con.commit()
        con.close()

def consulta_ligas(intento = 0):
    ''' Funcion recibe un valor de intento, en caso de fallar intenta hasta 3 veces'''
    intentos = 3 - intento
    list_ligas=[]
    if intentos >0:
        try:
            endpoint = "/leagues"
            respuesta= request_api(endpoint)
            ligas_api = respuesta["response"]  
            for liga in ligas_api:
                if liga["country"]["code"] == None:
                    liga["country"]["code"]="WM"
                                    
                id_liga = liga["league"]["id"]
                codigo_pais =  liga["country"]["code"]
                nombre_liga = liga["league"]["name"]
                tipo_liga = liga["league"]["type"]
                try:
                    ins = mysql_insert(ligas).values(id=id_liga,codigo_pais= codigo_pais, 
                                                     nombre_liga = nombre_liga, tipo_liga=tipo_liga )
                    ins = ins.on_duplicate_key_update(id=ins.inserted.id)  # Para MySQL, especificar que hacer en caso de duplicado
                    insertar_base(ins)
                except Exception as e:
                    # e es la variable que almacena información sobre la excepción ocurrida
                    print("Mensaje de error 2:", e)    
                    continue
            consulta_descripcion_ligas_api(ligas_api)
        except Exception as e:
            print("Mensaje de error 1:", e)
            time.sleep(0.5)
            consulta_ligas(intento+1)
    else:
        return list_ligas

In [14]:
#endpoint = "/teams?league=39&season=2023"
#respuesta= request_api(endpoint)
#ligas_api = respuesta["response"]  


In [15]:
# Busca id liga y temporada por pais
 
def buscar_liga_pais(codigo_pais: str):
      query_equipos_consulta = f"SELECT id, temporada from ligas a inner join datos_ligas\
            b on a.id = b.liga_id where codigo_pais = '{codigo_pais}'"

      id_ligas_consulta = consultaSQL(query_equipos_consulta,usuario,password,schema,host)

      return(id_ligas_consulta)

# Consulta a la API los equipos por liga y temporada
def consultar_equipos_por_liga_pais(id_ligas_consulta: list, limitar = True ):
      
      id = [ x for x in id_ligas_consulta[0].keys() if "id" in x][0]
      
      lista_respuestas_ligas=[]
      print(f"Cantidad de ligas a consultar: {len(id_ligas_consulta)}")
      conteo = 0

      # Agregar condicional, para buscar solo si no se buscó en el mismo día la combinación liga - temporada
      if limitar == True:
            for liga_temporada in id_ligas_consulta:

                  conteo += 1
                  id_liga = liga_temporada[f"{id}"]
                  temporada = liga_temporada["temporada"]

                  if f"{id_liga}&{temporada}" not in [i for i in glob.glob(f'teams_league={id}&season={temporada}*-{datetime.now().month}-{datetime.now().day}*.json')]:
                        endpoint = f"/teams?league={id_liga}&season={temporada}"
                        respuesta= request_api(endpoint)
                        lista_respuestas_ligas.append(respuesta)
                        time.sleep(6)
                  else:
                        with open(f"f'teams_league={id}&season={temporada}*-{datetime.now().month}-{datetime.now().day}*.json'") as file:
                              json_file = json.load(file)
                              lista_respuestas_ligas.append(json_file)
                       

                  print(f"Extraida liga {id_liga}, temporada {temporada}. [{conteo} / {len(id_ligas_consulta)}]")
      print("FIN CONSULTA EQUIPOS PAIS")
      return(lista_respuestas_ligas)

In [16]:
import time
# Inserta los equipos de todas las ligas de un país
def insertar_equipos_por_pais(codigo_pais:str):
    id_ligas_consulta = buscar_liga_pais(codigo_pais)
    lista_respuestas_ligas = consultar_equipos_por_liga_pais(id_ligas_consulta)
    for equipos_api in lista_respuestas_ligas:
        try:   
            id_liga = equipos_api["parameters"]["league"]

            for equipo in equipos_api["response"]:
                id_equipo = equipo["team"]["id"]
                nombre_equipo = equipo["team"]["name"]
                codigo_equipo = equipo["team"]["code"]
                fundado_equipo = equipo["team"]["founded"]
                try:
                    ins = mysql_insert(equipos).values(id=id_equipo,nombre= nombre_equipo,
                                                    codigo = codigo_equipo, fundado=fundado_equipo,
                                                    id_liga = id_liga
                                                        )
                    ins = ins.on_duplicate_key_update(id=ins.inserted.id)  # Para MySQL, especificar que hacer en caso de duplicado
                    insertar_base(ins)
                    #print("ok")
                except Exception as e:
                    # e es la variable que almacena información sobre la excepción ocurrida
                    print("Mensaje de error 2:", e)    
                    continue
        except Exception as e:
            # e es la variable que almacena información sobre la excepción ocurrida
            print("Mensaje de error 1:", e)    
            continue
    #cantidad_equipos = set(cantidad_equipos)


In [17]:
#insertar_equipos_por_pais("BO")

In [18]:
""" CREAR TABLA QUE TENGA
ID_LIGA, ID_EQUIPO, TEMPORADA 
para poder vincular a cada equipo con las ligas en las que juega"""

' CREAR TABLA QUE TENGA\nID_LIGA, ID_EQUIPO, TEMPORADA \npara poder vincular a cada equipo con las ligas en las que juega'

In [19]:
#para el equipo consultado buscar que ligas hay en su pais y continente
#endpoint = f"/teams?statistics?season={temporada}&team={id_equipo}&league={id_liga}"
#endpoint = f"/teams?country=Argentina"
#respuesta= request_api(endpoint)


In [20]:
# Crear tabla en mysql que tenga id_equipo, id_liga, temporada
# Seleccionar a los primeros 38 ligas con sus respectivas temporadas
# Para cada combinación, consultar por equipos y guardar: id_equipo, id_liga, temporada

def buscar_liga_temporada_internacionales():
    query = "select liga_id, temporada from futbol.datos_ligas where liga_id <= 38;"

    id_ligas_consulta = consultaSQL(query,usuario,password,schema,host)

    return(id_ligas_consulta)



In [20]:
#ligas_temporadas = buscar_liga_temporada_internacionales()

In [21]:
def insertar_equipos_ligas(codigo_pais:str):
    if codigo_pais == "WM":
        id_ligas_consulta = buscar_liga_temporada_internacionales()
    else:
         id_ligas_consulta = buscar_liga_pais(codigo_pais)
         
    lista_respuestas_ligas = consultar_equipos_por_liga_pais(id_ligas_consulta)

    for liga_temp in lista_respuestas_ligas:
            try:   
                id_liga = liga_temp["parameters"]["league"]
                temporada = liga_temp["parameters"]["season"]
                for liga in liga_temp["response"]:
                    id_equipo = liga["team"]["id"]
                    nombre_equipo = liga["team"]["name"]
                    codigo_equipo = liga["team"]["code"]
                    fundado_equipo = liga["team"]["founded"]
                    #print(id_liga, temporada, id_equipo, nombre_equipo)
                    try:
                        # Primero se inserta en tabla equipos
                        ins = mysql_insert(equipos).values(id=id_equipo,nombre= nombre_equipo,
                                codigo = codigo_equipo, fundado=fundado_equipo,
                                id_liga = id_liga
                                    )
                        ins = ins.on_duplicate_key_update(id=ins.inserted.id)  # Para MySQL, especificar que hacer en caso de duplicado
                        insertar_base(ins)
                        # Luego se inserta en ligas_equipos
                        ins = mysql_insert(ligas_equipos).values(
                                                        id_liga = id_liga, id_equipo = id_equipo,
                                                        temporada=temporada,nombre_equipo= nombre_equipo,
                                                            )
                        #ins = ins.on_duplicate_key_update(id=ins.inserted.id)  # Para MySQL, especificar que hacer en caso de duplicado
                        insertar_base(ins)
                        #print("ok")
                    except Exception as e:
                        # e es la variable que almacena información sobre la excepción ocurrida
                        print("Mensaje de error 2:", e)    
                        continue
            except Exception as e:
                # e es la variable que almacena información sobre la excepción ocurrida
                print("Mensaje de error 1:", e)                
                continue

In [22]:
#insertar_equipos_ligas("AR")

### Buscar estadísticas de equipo por liga y temporada

In [38]:
#Buscar id de equipo en base (utilizando una consulta sql con like) -> devolver opciones de equipos con respectivo id

In [22]:
 
def buscar_equipo(nombre_equipo: str):
      query_equipos_consulta = f"SELECT id, nombre, id_liga from futbol.equipos\
            where nombre like '%{nombre_equipo}%'"

      respuesta = consultaSQL(query_equipos_consulta,usuario,password,schema,host)

      return(id_ligas_consulta)

In [23]:
query_equipos_consulta = f"SELECT id, nombre, id_liga from futbol.equipos\
            where nombre like '%San Lorenzo%'"

respuesta = consultaSQL(query_equipos_consulta,usuario,password,schema,host)

In [24]:
id_equipo = input("Ingrese id equipo:")

In [1]:
print("Su consulta tiene las siguientes opciones:\n")
for registro in respuesta:
    print(f"Equipo: {registro['nombre']}    |    id: {registro['id']} |  id liga: {registro['id_liga']}")
id_equipo = input("Ingrese id equipo:")

id_ligas = []
for registro in respuesta:
    if registro["id"] == id_equipo:
        id_ligas.append(registro["id_liga"])

id_liga = input(f"Ingrese alguna de las siguientes ligas: {' | '.join(id_ligas)})")




Su consulta tiene las siguientes opciones:



NameError: name 'respuesta' is not defined

liga